# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from tqdm import tqdm
from time import time

In [2]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

#X_val = np.load('data/x_val.npy')
#y_val = np.load('data/y_val.npy')

In [3]:
#X_train = np.concatenate((X_train, X_val), axis=0)
#y_train = np.concatenate((y_train, y_val), axis=0)

# Playground

In [4]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 2142,  1863, 13082, 35620, 19397, 47097, 74400, 11189,  1209,
          139]))

6 is normal class

In [5]:
normal_data = X_train[y_train == 6] # normal class

In [6]:
np.unique(y_test, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([  535,   466,  3271,  8905,  4849, 11774, 18600,  2798,   302,
           35]))

In [7]:
len(normal_data)

74400

# Helpers

In [8]:
def evaluate_model(model, y_test, y_pred):
    positive_class = 'normal'
    
    accuracy = accuracy_score(y_test, y_pred)
    
    precision = precision_score(y_test, y_pred, pos_label=positive_class)
    recall = recall_score(y_test, y_pred, pos_label=positive_class)
    f1 = f1_score(y_test, y_pred, pos_label=positive_class)
    
    metrics = {
        'model': [model],
        'accuracy': [accuracy],
        'precision': [precision],
        'recall': [recall],
        'f1': [f1],
    }
    
    return metrics

# SGD-OCSVM

In [9]:
from sklearn.linear_model import SGDOneClassSVM

In [74]:
sgd_ocsvm_start = time()

sgd_ocsvm = SGDOneClassSVM(nu=0.03, max_iter=1000, tol=0.001, learning_rate='invscaling', eta0=0.1, random_state=42)
sgd_ocsvm.fit(normal_data)
        
print(f"seconds: {time() - sgd_ocsvm_start}")

seconds: 0.09594535827636719


In [75]:
y_pred_test_sgd_ocsvm = sgd_ocsvm.predict(X_test)
y_pred_test_sgd_ocsvm = np.where(y_pred_test_sgd_ocsvm == 1, 'normal', 'anomaly')
y_test_converted_sgd_ocsvm = np.where(y_test == 6, 'normal', 'anomaly')

In [76]:
sgd_ocsvm_df = pd.DataFrame(evaluate_model("sgd_ocsvm", y_test_converted_sgd_ocsvm, y_pred_test_sgd_ocsvm))
sgd_ocsvm_df

,model,accuracy,precision,recall,f1
0,sgd_ocsvm,0.790861,0.636815,0.97871,0.771585


# PCA Reconstruction

In [77]:
from sklearn.decomposition import PCA

In [111]:
pca_start = time()

pca = PCA(n_components=30, whiten=True, svd_solver='auto')

normal_data_pca = pca.fit_transform(normal_data)
normal_data_reconstructed = pca.inverse_transform(normal_data_pca)

reconstruction_error = np.mean((normal_data - normal_data_reconstructed) ** 2, axis=1)

threshold = np.percentile(reconstruction_error, 95)

X_test_pca = pca.transform(X_test)
X_test_reconstructed = pca.inverse_transform(X_test_pca)
X_test_reconstruction_error = np.mean((X_test - X_test_reconstructed) ** 2, axis=1)

print(f"seconds: {time() - pca_start}")

seconds: 1.5492980480194092


In [112]:
y_pred_test_pca = np.where(X_test_reconstruction_error > threshold, 'anomaly', 'normal')
y_test_converted_pca = np.where(y_test == 6, 'normal', 'anomaly')

In [113]:
pca_df = pd.DataFrame(evaluate_model('PCA Reconstruction', y_test_converted_pca, y_pred_test_pca))
pca_df

,model,accuracy,precision,recall,f1
0,PCA Reconstruction,0.795576,0.647749,0.950484,0.770444


# Results (2)

In [114]:
results_df = pd.concat([sgd_ocsvm_df, pca_df], ignore_index=True)
results_df.to_csv('results/results_unsw_2.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,sgd_ocsvm,0.790861,0.636815,0.978710,0.771585
1,PCA Reconstruction,0.795576,0.647749,0.950484,0.770444


# OCSVM

In [45]:
from sklearn.svm import OneClassSVM

In [106]:
ocsvm = OneClassSVM(kernel='rbf', gamma=0.1, nu=0.5)

batch_size = 50000
n_batches = 1# int(np.ceil(len(normal_data) / batch_size))

for i in tqdm(range(n_batches), desc="Training OCSVM"):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(normal_data))
    batch_data = normal_data[start:end]
    
    if i == 0:
        ocsvm.fit(batch_data)
    else:
        ocsvm.fit(np.vstack((ocsvm.support_vectors_, batch_data)))

Training OCSVM: 100%|██████████| 1/1 [02:04<00:00, 124.83s/it]


In [107]:
y_pred_test_ocsvm = ocsvm.predict(X_test)
y_pred_test_ocsvm = np.where(y_pred_test_ocsvm == 1, 'normal', 'anomaly')
y_test_converted_ocsvm = np.where(y_test == 6, 'normal', 'anomaly')

In [108]:
ocsvm_df = pd.DataFrame(evaluate_model("ocsvm", y_test_converted_ocsvm, y_pred_test_ocsvm))
ocsvm_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.765887,0.760379,0.513011,0.612668


# LOF

In [121]:
from sklearn.neighbors import LocalOutlierFactor

In [122]:
lof_start = time()

lof = LocalOutlierFactor(n_neighbors=25, contamination=0.1, novelty=True)
lof.fit(X_train)

print(f"seconds: {time() - lof_start}")

seconds: 26.28725290298462


In [123]:
y_pred_test_lof = lof.predict(X_test)
y_pred_test_lof = np.where(y_pred_test_lof == 1, 'normal', 'anomaly')
y_test_converted_lof = np.where(y_test == 6, 'normal', 'anomaly')

In [124]:
lof_df = pd.DataFrame(evaluate_model("lof", y_test_converted_lof, y_pred_test_lof)) 
lof_df

,model,accuracy,precision,recall,f1
0,lof,0.426293,0.381883,0.953065,0.545278


# IF

In [125]:
from sklearn.ensemble import IsolationForest

In [154]:
if_start = time()

iforest = IsolationForest(n_estimators=400, max_samples=5192, contamination=0.1, random_state=42)
iforest.fit(X_train)

print(f"seconds: {time() - if_start}")

seconds: 44.988903522491455


In [155]:
y_pred_test_if = iforest.predict(X_test)
y_pred_test_if = np.where(y_pred_test_if == 1, 'normal', 'anomaly')
y_test_converted_if = np.where(y_test == 6, 'normal', 'anomaly')

In [157]:
if_df = pd.DataFrame(evaluate_model("if", y_test_converted_if, y_pred_test_if))
if_df

,model,accuracy,precision,recall,f1
0,if,0.34625,0.337483,0.842419,0.481908


# Results

In [158]:
results_df = pd.concat([ocsvm_df, lof_df, if_df], ignore_index=True)
results_df.to_csv('results/results_unsw.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.765887,0.760379,0.513011,0.612668
1,lof,0.426293,0.381883,0.953065,0.545278
2,if,0.346250,0.337483,0.842419,0.481908
